In [1]:
import sys
sys.path.insert(0, r'/Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/build')

import numpy as np

import cpp_probabilities

# Basic testing

In [26]:
import random

# Assuming you have some function to create your grid data
def create_your_grid_data(num_points):
    # Replace this with your actual grid data creation logic
    grid_x, grid_y = np.meshgrid(np.linspace(0, 1, num_points), np.linspace(0, 1, num_points))
    grid_density_0 = np.random.rand(num_points, num_points)  # Placeholder values
    grid_density_1 = np.random.rand(num_points, num_points)  # Placeholder values
    return grid_x, grid_y, grid_density_0, grid_density_1

def create_specific_grid_data(num_points):
    # Create a meshgrid
    grid_x, grid_y = np.meshgrid(np.linspace(0, 1, num_points), np.linspace(0, 1, num_points))

    # Create grid densities: all zeros for grid_density_0 and all 0.5s for grid_density_1
    grid_density_0 = np.zeros((num_points, num_points))
    grid_density_1 = np.full((num_points, num_points), 0.5)

    return grid_x, grid_y, grid_density_0, grid_density_1


# Create instances of GridData for each qubit
kde_grid_dict = {}
for qubit_idx in range(2):  # Assuming two qubits
    grid_x, grid_y, grid_density_0, grid_density_1 = create_specific_grid_data(100)

    kde_grid_dict[qubit_idx] = cpp_probabilities.GridData(grid_x, grid_y, grid_density_0, grid_density_1)


def generate_random_qubit_mapping(num_keys):
    """
    Generates a random qubit mapping.

    Args:
    num_keys (int): The number of keys in the mapping.

    Returns:
    dict: A dictionary where keys are mapped randomly to either 0 or 1.
    """
    qubit_mapping = {}
    for i in range(num_keys):
        qubit_mapping[i] = random.randint(0, 1)
    return qubit_mapping

# Example usage
len_IQ = 9   # Number of keys in the mapping
qubit_mapping = generate_random_qubit_mapping(len_IQ)
print(qubit_mapping)


{0: 1, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 1, 7: 0, 8: 0}


In [27]:
# Example NumPy array (replace this with your actual data)
scaled_IQ_data_np = np.random.rand(int(1e2), len_IQ*2)  # 10 shots, len_IQ columns (2 measurements per shot)

# Convert NumPy array to Eigen::MatrixXd
scaled_IQ_data = cpp_probabilities.numpy_to_eigen(scaled_IQ_data_np)


# Example number of syndrome rounds
synd_rounds = 3

# Call the get_counts function
counts = cpp_probabilities.get_counts(scaled_IQ_data, qubit_mapping, kde_grid_dict, synd_rounds)
print(sorted(counts.items(), key=lambda x: x[1], reverse=True))


[('111 11 11 11', 100)]
